In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def toUpperNoSpaces(row,column):
    return str(row[column]).replace(" ","").upper().strip()


dfAll = pd.read_csv('netflixList.csv')
dfOM = pd.read_csv('netflixOM.csv')
dfOS = pd.read_csv('netflixOS.csv')

#dfOS = pd.read_csv('Downloads/project/netflixOS.csv')
# operations made to compare the column title
dfOM['TitleToCompare'] = dfOM.apply(lambda row: toUpperNoSpaces(row,'Title'), axis = 1)
dfOS['TitleToCompare'] = dfOS.apply(lambda row: toUpperNoSpaces(row,'Title'), axis = 1)
dfAll['TitleToCompare'] = dfAll.apply(lambda row: toUpperNoSpaces(row,'Title'), axis = 1)

dfAll['GenreToCompare'] = dfAll.apply(lambda row: toUpperNoSpaces(row,'Genre'), axis = 1)

# add new column: Original, 1: is produced by netflix, 0: is not produced by netflix
df = dfAll.assign(Original = dfAll.TitleToCompare.isin(pd.concat([dfOM.TitleToCompare,dfOS.TitleToCompare])).astype(int))

#  ------------ columns ------------------------
#Index(['Title', 'Genre', 'Tags', 'Languages', 'Series or Movie',
#       'Hidden Gem Score', 'Country Availability', 'Runtime', 'Director',
#       'Writer', 'Actors', 'View Rating', 'IMDb Score',
#       'Rotten Tomatoes Score', 'Metacritic Score', 'Awards Received',
#       'Awards Nominated For', 'Boxoffice', 'Release Date',
#       'Netflix Release Date', 'Production House', 'Netflix Link', 'IMDb Link',
#       'Summary', 'IMDb Votes', 'Image', 'Poster', 'TMDb Trailer',
#       'Trailer Site', 'TitleToCompare', 'Original'],
#      dtype='object')

# How many movies and series has netflix produced in 2021. 1: Netlfix original

# Recognition Indicator
df['Year'] = pd.to_numeric(df['Release Date'].str[-4:])
dfAwardsSeries = pd.DataFrame()

filterOriginal = df['Original'] == 1

for year in range(2010, 2022):
    filterYear = df['Year'] == year
    filterSM = df['Series or Movie'].str.contains(pat = 'Serie')
    dfTemp = df[['Title','Awards Received','Series or Movie','Original','Year','Genre']].where(filterYear & filterOriginal & filterSM, inplace = False).dropna().drop_duplicates()
    # finding the genre with most awards received
    dfFirstGenre = dfTemp[['Awards Received','Genre']].groupby(['Genre']).max().sort_values(by='Awards Received', ascending=False).head(1).reset_index()
    lgenre = 'No genre'
    if dfFirstGenre.size > 0:
        lgenre = dfFirstGenre['Genre'].iloc[0]
    dfAwardsSeries = dfAwardsSeries.append(dfTemp.groupby(['Series or Movie']).agg(Films = ('Title', 'count'), Y = ('Awards Received', 'sum')).assign(X = year, Genre_most_awards = lgenre))

dfAwardsSeries[['X','Y','Films','Genre_most_awards']].to_csv(r'D:\COMPUTER SCIENCE\2 SEMESTER\information visualization\project\series_awards.csv', index = False, header=True)

dfAwardsSeries

#dfAwards.sort_values(['Series or Movie','Year'], ascending=[False, True], inplace=False)

#ft = df['Title'].str.contains(pat = 'Orange')
#df5 = df[['Title','Series or Movie','Release Date','Awards Nominated For']].where(ft, inplace = False).dropna().drop_duplicates()#.groupby(['Series or Movie']).agg(Films = ('Title', 'count'), Awards_Received = ('Awards Received', 'sum'), Awards_Nominated = ('Awards Nominated For', 'sum'))
#df5


,Films,Y,X,Genre_most_awards
Series or Movie,,,,
Series,1,2.0,2010,"Crime, Horror, Thriller"
Series,1,3.0,2011,"Animation, Adventure, Family"
Series,1,6.0,2012,"Animation, Action, Adventure, Comedy, Family, ..."
Series,2,85.0,2013,"Comedy, Crime, Drama"
Series,5,56.0,2014,"Animation, Comedy, Drama"
Series,11,75.0,2015,Comedy
Series,15,193.0,2016,"Biography, Drama, History"
Series,22,106.0,2017,"Adventure, Comedy, Drama, Family, Mystery"
Series,28,85.0,2018,Reality-TV


In [62]:
dfAwardsMovies = pd.DataFrame()

for year in range(2010, 2022):
    filterYear = df['Year'] == year
    filterSM = df['Series or Movie'].str.contains(pat = 'Movie')
    dfTemp = df[['Title','Awards Received','Series or Movie','Original','Year','Genre']].where(filterYear & filterOriginal & filterSM, inplace = False).dropna().drop_duplicates()
    # finding the genre with most awards received
    dfFirstGenre = dfTemp[['Awards Received','Genre']].groupby(['Genre']).max().sort_values(by='Awards Received', ascending=False).head(1).reset_index()
    lgenre = 'No genre'
    if dfFirstGenre.size > 0:
        lgenre = dfFirstGenre['Genre'].iloc[0]
    dfAwardsMovies = dfAwardsMovies.append(dfTemp.groupby(['Series or Movie']).agg(Films = ('Title', 'count'), Y = ('Awards Received', 'sum')).assign(Genre_most_awards = lgenre, X = year))

dfAwardsMovies[['X','Y','Films','Genre_most_awards']].to_csv(r'D:\COMPUTER SCIENCE\2 SEMESTER\information visualization\project\movies_awards.csv', index = False, header=True)

dfAwardsMovies

,Films,Y,Genre_most_awards,X
Series or Movie,,,,
Movie,1,1.0,Drama,2010
Movie,1,80.0,"Crime, Drama",2011
Movie,1,1.0,Comedy,2012
Movie,2,5.0,"Documentary, Sport",2013
Movie,4,51.0,"Documentary, War",2014
Movie,6,58.0,"Drama, War",2015
Movie,11,70.0,"Documentary, Crime, History, News",2016
Movie,19,76.0,"Action, Crime, Drama",2017
Movie,31,396.0,Drama,2018


In [133]:
# count films, average ranking by genre, to see which genre has more rating in "Movies" (Original content)
# get unique values from Genre
dfTemp = df['GenreToCompare'].drop_duplicates().str.split(',', expand=True)
dfGenre = dfTemp[0].drop_duplicates()
for col in dfTemp.columns:
    dfGenre = dfGenre.append(dfTemp[col].drop_duplicates(), ignore_index=True)
dfGenre = dfGenre.drop_duplicates().dropna().drop(dfGenre.index[10])

#dfGenre

# get unique values from country availability
dfTemp2 = df['Country Availability'].drop_duplicates().str.split(',', expand=True)
dfCountries = dfTemp2[0].drop_duplicates()
for col in dfTemp2.columns:
    dfCountries = dfCountries.append(dfTemp2[col].drop_duplicates(), ignore_index=True)
dfCountries = dfCountries.drop_duplicates().dropna()

# count films, average ranking by genre, to see which genre has more rating in "Movies" (Original content)
dfGenreFilms = pd.DataFrame()
dfFilms = pd.DataFrame()
for countries in dfCountries:
    filterTmp2 = df['Country Availability'].str.contains(pat = countries)
    dfFilms = dfFilms.append(df[['GenreToCompare','Original','Title','IMDb Score','Series or Movie','Country Availability']].where(filterTmp2 & filterOriginal, inplace = False).dropna().drop_duplicates().assign(Country = countries))

for genre in dfGenre:
    filterSM = dfFilms['Series or Movie'].str.contains(pat = 'Movie')
    filterTemp = dfFilms['GenreToCompare'].str.contains(pat = genre)
    dfGenreFilms = dfGenreFilms.append(dfFilms[['GenreToCompare','IMDb Score','Series or Movie','Country']].where(filterTemp & filterSM, inplace = False).dropna().groupby(['Country','Series or Movie']).agg(Films = ('GenreToCompare', 'count'), IMDb_Score = ('IMDb Score', 'mean')).assign(Genre = genre))

dfGenreFilms = dfGenreFilms.reset_index()#.sort_values(['Country','Genre'], ascending=[True, True], inplace=True)
dfGenreFilms = dfGenreFilms[['Country','Films','IMDb_Score','Genre']]

# Filling all genres with "0" values in case they don't exist
for genre in dfGenre:
    filterTemp = dfGenreFilms['Genre'].str.contains(pat = genre)
    for countries in dfCountries:
        filterTmp2 = dfGenreFilms['Country'].str.contains(pat = countries)
        dfTemp3 = dfGenreFilms.where(filterTemp & filterTmp2, inplace = False).dropna()
        if dfTemp3.size < 1:
            dfGenreFilms = dfGenreFilms.append({'Country': countries, 'Films': 0, 'IMDb_Score': 0.0, 'Genre': genre}, ignore_index = True)

dfGenreFilms.sort_values(['Country','Genre'], ascending=[True, True], inplace=True)
dfGenreFilms.to_csv(r'D:\COMPUTER SCIENCE\2 SEMESTER\information visualization\project\Movies_genre.csv', index = False, header=True)    
dfGenreFilms

,Country,Films,IMDb_Score,Genre
180,Argentina,49,5.977551,ACTION
1005,Argentina,0,0.000000,ADULT
216,Argentina,25,6.128000,ADVENTURE
108,Argentina,7,7.000000,ANIMATION
324,Argentina,40,6.847500,BIOGRAPHY
...,...,...,...,...
755,United States,20,6.970000,SPORT
917,United States,0,0.000000,TALK-SHOW
323,United States,92,5.714130,THRILLER
791,United States,15,6.993333,WAR


In [134]:
# count films, average ranking by genre, to see which genre has more rating in "Series" (Original content)
dfGenreSeries = pd.DataFrame()

for genre in dfGenre:
    filterSM = dfFilms['Series or Movie'].str.contains(pat = 'Serie')
    filterTemp = dfFilms['GenreToCompare'].str.contains(pat = genre)
    dfGenreSeries = dfGenreSeries.append(dfFilms[['GenreToCompare','IMDb Score','Series or Movie','Country']].where(filterTemp & filterSM, inplace = False).dropna().groupby(['Country','Series or Movie']).agg(Films = ('GenreToCompare', 'count'), IMDb_Score = ('IMDb Score', 'mean')).assign(Genre = genre))

dfGenreSeries = dfGenreSeries.reset_index()
dfGenreSeries = dfGenreSeries[['Country','Films','IMDb_Score','Genre']]

# Filling all genres with "0" values in case they don't exist
for genre in dfGenre:
    filterTemp = dfGenreSeries['Genre'].str.contains(pat = genre)
    for countries in dfCountries:
        filterTmp2 = dfGenreSeries['Country'].str.contains(pat = countries)
        dfTemp3 = dfGenreSeries.where(filterTemp & filterTmp2, inplace = False).dropna()
        if dfTemp3.size < 1:
            dfGenreSeries = dfGenreSeries.append({'Country': countries, 'Films': 0, 'IMDb_Score': 0.0, 'Genre': genre}, ignore_index = True)

dfGenreSeries.sort_values(['Country','Genre'], ascending=[True, True], inplace=True)
dfGenreSeries.to_csv(r'D:\COMPUTER SCIENCE\2 SEMESTER\information visualization\project\Series_genre.csv', index = False, header=True)    
dfGenreSeries

,Country,Films,IMDb_Score,Genre
180,Argentina,42,7.076190,ACTION
1005,Argentina,0,0.000000,ADULT
216,Argentina,48,6.968750,ADVENTURE
108,Argentina,67,6.940299,ANIMATION
324,Argentina,8,8.012500,BIOGRAPHY
...,...,...,...,...
827,United States,9,7.422222,SPORT
719,United States,11,6.863636,TALK-SHOW
323,United States,41,7.173171,THRILLER
863,United States,2,7.500000,WAR


In [24]:
# how much content the director and writer produced, and what’s their Highest/lowest score
df[['Directoqr', 'IMDb Score', 'Rotten Tomatoes Score', 'Metacritic Score']].groupby(['Director']).agg(count = ('Director', 'count'), IMDb_Max = ('IMDb Score', 'max'), IMDb_Min = ('IMDb Score', 'min'))


,count,IMDb_Max,IMDb_Min
Director,,,
A. Rajdeep,1,3.2,3.2
A. Salaam,1,4.1,4.1
A.L. Vijay,2,6.1,5.3
A.R. Murugadoss,3,6.8,6.6
Aamir Bashir,1,6.4,6.4
...,...,...,...
Óscar Menéndez,1,NaN,NaN
Ömer Faruk Sorak,2,7.4,7.3
Ömer Ugur,2,4.8,4.6
